# Capstone Project
## Machine Learning Engineer Nanodegree
**Calvin Ku**

**July 15th, 2016**

## Definition
(approximately 1 - 2 pages)

### Project Overview
In this section, look to provide a high-level overview of the project in layman’s terms. Questions to ask yourself when writing this section:

* Has an overview of the project been provided, such as the problem domain, project origin, and related datasets or input data?
* Has enough background information been given so that an uninformed reader would understand the problem domain and following problem statement?

### Problem Statement

In this section, you will want to clearly define the problem that you are trying to solve, including the strategy (outline of tasks) you will use to achieve the desired solution. You should also thoroughly discuss what the intended solution will be for this problem. Questions to ask yourself when writing this section:

* Is the problem statement clearly defined? Will the reader understand what you are expecting to solve?
* Have you thoroughly discussed how you will attempt to solve the problem?
* Is an anticipated solution clearly defined? Will the reader understand what results you are looking for?

### Metrics
In this section, you will need to clearly define the metrics or calculations you will use to measure performance of a model or result in your project. These calculations and metrics should be justified based on the characteristics of the problem and problem domain. Questions to ask yourself when writing this section:

* Are the metrics you’ve chosen to measure the performance of your models clearly discussed and defined?
* Have you provided reasonable justification for the metrics chosen based on the problem and solution?

# Capstone Project
## Machine Learning Engineer Nanodegree
**Calvin Ku**

**July 15th, 2016**

## Definition
(approximately 1 - 2 pages)

### Project Overview
In this section, look to provide a high-level overview of the project in layman’s terms. Questions to ask yourself when writing this section:

* Has an overview of the project been provided, such as the problem domain, project origin, and related datasets or input data?
* Has enough background information been given so that an uninformed reader would understand the problem domain and following problem statement?

### Problem Statement

In this section, you will want to clearly define the problem that you are trying to solve, including the strategy (outline of tasks) you will use to achieve the desired solution. You should also thoroughly discuss what the intended solution will be for this problem. Questions to ask yourself when writing this section:

* Is the problem statement clearly defined? Will the reader understand what you are expecting to solve?
* Have you thoroughly discussed how you will attempt to solve the problem?
* Is an anticipated solution clearly defined? Will the reader understand what results you are looking for?

### Metrics
In this section, you will need to clearly define the metrics or calculations you will use to measure performance of a model or result in your project. These calculations and metrics should be justified based on the characteristics of the problem and problem domain. Questions to ask yourself when writing this section:

* Are the metrics you’ve chosen to measure the performance of your models clearly discussed and defined?
* Have you provided reasonable justification for the metrics chosen based on the problem and solution?

## Analysis
(approximately 2 - 4 pages)

### Data Exploration
In this section, you will be expected to analyze the data you are using for the problem. This data can either be in the form of a dataset (or datasets), input data (or input files), or even an environment. The type of data should be thoroughly described and, if possible, have basic statistics and information presented (such as discussion of input features or defining characteristics about the input or environment). Any abnormalities or interesting qualities about the data that may need to be addressed have been identified (such as features that need to be transformed or the possibility of outliers). Questions to ask yourself when writing this section:

* If a dataset is present for this problem, have you thoroughly discussed certain features about the dataset? Has a data sample been provided to the reader?
* If a dataset is present for this problem, are statistics about the dataset calculated and reported? Have any relevant results from this calculation been discussed?
* If a dataset is not present for this problem, has discussion been made about the input space or input data for your problem?
* Are there any abnormalities or characteristics about the input space or dataset that need to be addressed? (categorical variables, missing values, outliers, etc.)

### Exploratory Visualization
In this section, you will need to provide some form of visualization that summarizes or extracts a relevant characteristic or feature about the data. The visualization should adequately support the data being used. Discuss why this visualization was chosen and how it is relevant. Questions to ask yourself when writing this section:

* Have you visualized a relevant characteristic or feature about the dataset or input data?
* Is the visualization thoroughly analyzed and discussed?
* If a plot is provided, are the axes, title, and datum clearly defined?

### Algorithms and Techniques
In this section, you will need to discuss the algorithms and techniques you intend to use for solving the problem. You should justify the use of each one based on the characteristics of the problem and the problem domain. Questions to ask yourself when writing this section:

* Are the algorithms you will use, including any default variables/parameters in the project clearly defined?
* Are the techniques to be used thoroughly discussed and justified?
* Is it made clear how the input data or datasets will be handled by the algorithms and techniques chosen?

### Benchmark
In this section, you will need to provide a clearly defined benchmark result or threshold for comparing across performances obtained by your solution. The reasoning behind the benchmark (in the case where it is not an established result) should be discussed. Questions to ask yourself when writing this section:

* Has some result or value been provided that acts as a benchmark for measuring performance?
* Is it clear how this result or value was obtained (whether by data or by hypothesis)?

## Methodology
(approximately 3 - 5 pages)

### Data Preprocessing
In this section, all of your preprocessing steps will need to be clearly documented, if any were necessary. From the previous section, any of the abnormalities or characteristics that you identified about the dataset will be addressed and corrected here. Questions to ask yourself when writing this section:

* If the algorithms chosen require preprocessing steps like feature selection or feature transformations, have they been properly documented?
* Based on the Data Exploration section, if there were abnormalities or characteristics that needed to be addressed, have they been properly corrected?
* If no preprocessing is needed, has it been made clear why?

### Implementation
In this section, the process for which metrics, algorithms, and techniques that you implemented for the given data will need to be clearly documented. It should be abundantly clear how the implementation was carried out, and discussion should be made regarding any complications that occurred during this process. Questions to ask yourself when writing this section:

* Is it made clear how the algorithms and techniques were implemented with the given datasets or input data?
* Were there any complications with the original metrics or techniques that required changing prior to acquiring a solution?
* Was there any part of the coding process (e.g., writing complicated functions) that should be documented?

### Refinement
In this section, you will need to discuss the process of improvement you made upon the algorithms and techniques you used in your implementation. For example, adjusting parameters for certain models to acquire improved solutions would fall under the refinement category. Your initial and final solutions should be reported, as well as any significant intermediate results as necessary. Questions to ask yourself when writing this section:

* Has an initial solution been found and clearly reported?
* Is the process of improvement clearly documented, such as what techniques were used?
* Are intermediate and final solutions clearly reported as the process is improved?

## Results
(approximately 2 - 3 pages)

### Model Evaluation and Validation
In this section, the final model and any supporting qualities should be evaluated in detail. It should be clear how the final model was derived and why this model was chosen. In addition, some type of analysis should be used to validate the robustness of this model and its solution, such as manipulating the input data or environment to see how the model’s solution is affected (this is called sensitivity analysis). Questions to ask yourself when writing this section:

* Is the final model reasonable and aligning with solution expectations? Are the final parameters of the model appropriate?
* Has the final model been tested with various inputs to evaluate whether the model generalizes well to unseen data?
* Is the model robust enough for the problem? Do small perturbations (changes) in training data or the input space greatly affect the results?
* Can results found from the model be trusted?

### Justification
In this section, your model’s final solution and its results should be compared to the benchmark you established earlier in the project using some type of statistical analysis. You should also justify whether these results and the solution are significant enough to have solved the problem posed in the project. Questions to ask yourself when writing this section:

* Are the final results found stronger than the benchmark result reported earlier?
* Have you thoroughly analyzed and discussed the final solution?
* Is the final solution significant enough to have solved the problem?

## Conclusion
(approximately 1 - 2 pages)

### Free-Form Visualization
In this section, you will need to provide some form of visualization that emphasizes an important quality about the project. It is much more free-form, but should reasonably support a significant result or characteristic about the problem that you want to discuss. Questions to ask yourself when writing this section:

* Have you visualized a relevant or important quality about the problem, dataset, input data, or results?
* Is the visualization thoroughly analyzed and discussed?
* If a plot is provided, are the axes, title, and datum clearly defined?

### Reflection
In this section, you will summarize the entire end-to-end problem solution and discuss one or two particular aspects of the project you found interesting or difficult. You are expected to reflect on the project as a whole to show that you have a firm understanding of the entire process employed in your work. Questions to ask yourself when writing this section:

* Have you thoroughly summarized the entire process you used for this project?
* Were there any interesting aspects of the project?
* Were there any difficult aspects of the project?
* Does the final model and solution fit your expectations for the problem, and should it be used in a general setting to solve these types of problems?

### Improvement
In this section, you will need to provide discussion as to how one aspect of the implementation you designed could be improved. As an example, consider ways your implementation can be made more general, and what would need to be modified. You do not need to make this improvement, but the potential solutions resulting from these changes are considered and compared/contrasted to your current solution. Questions to ask yourself when writing this section:

* Are there further improvements that could be made on the algorithms or techniques you used in this project?
* Were there algorithms or techniques you researched that you did not know how to implement, but would consider using if you knew how?
* If you used your final solution as the new benchmark, do you think an even better solution exists?

## Before submitting your report, ask yourself…
* Does the project report you’ve written follow a well-organized structure similar to that of the project template?
* Is each section (particularly Analysis and Methodology) written in a clear, concise and specific fashion? Are there any ambiguous terms or phrases that need clarification?
* Would the intended audience of your project be able to understand your analysis, methods, and results?
* Have you properly proof-read your project report to assure there are minimal grammatical and spelling mistakes?
* Are all the resources used for this project correctly cited and referenced?
* Is the code that implements your solution easily readable and properly commented?
* Does the code execute without error and produce results similar to those reported?

In [2]:
from __future__ import division

%matplotlib inline
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
from os import getcwd
from pandas_datareader import data as web
from pandas_datareader._utils import RemoteDataError
from IPython.display import display

stock_data_path = 'stock_data/'

def get_max_close(symbol):
    """Return the maximum closing value for stock indicated by symbol.
    Note: Data for a stock is stored in file stock_data/<symbol>.csv
    """
    
    df = pd.read_csv(stock_data_path + "{}.csv".format(symbol))
    return df['Close'].max()

def test_run():
    """Functoin called by Test Run
    """
    # Get SP500 components for all years
    df_sp500 = pd.read_csv('sp500.csv')
    
    # # Get SP500 (2009) components
    ix2009 = df_sp500['2009'] == 'X'
    df_sp500_2009 = df_sp500['Ticker'][ix2009]
#     df_sp500_2009 = df_sp500_2009.drop([])
    
    # Stocks to be dropped
    stock_goner = ['AGN (Old)', 'AYE', 'BJS', 'BNI', 'CEPH', 'COV', 'DTV', 'EK', 'EP', 'EQ', 'FDO', 'GENZ', 'HCBK', 'HSP', \
                  'JAVA', 'LO', 'MFE', 'MHS', 'MI', 'MWV', 'NOVL', 'PBG', 'PLL', 'PTV', 'RSH', 'SIAL', 'SWY', \
                  'TEG', 'WYE', 'XTO']
    
    # Drop dead stocks
    df_sp500_2009 = df_sp500_2009[-df_sp500_2009.isin(stock_goner)]
    
    # Add SPY and GOOG
    df_sp500_2009 = df_sp500_2009.append(pd.Series(['GOOG', 'SPY']))
    
    # Reindex/Recolumn/Sort SP500 (2009)
#     print(range(df_sp500_2009.shape[0]))
    df_sp500_2009 = df_sp500_2009.sort_values()
    df_sp500_2009.index = range(df_sp500_2009.shape[0])   
#     display(df_sp500_2009)
    
    dfSPY500_2009 = pd.read_csv('sp500_2009.csv', header=None, usecols = [1])
    dfSPY500_2009.columns = ['Symbol']
    
#     Create an empty dataframe
    start_date = '2009-01-01'
    end_date = '2016-06-28'
    
    dates = pd.date_range(start_date, end_date)
    df1 = pd.DataFrame(index=dates)
    
#     # Read GOOG into the temp dataframe
    dfGOOG = pd.read_csv(stock_data_path + "GOOG.csv", index_col="Date", parse_dates=True, na_values=['nan'])
    df1 = df1.join(dfGOOG)
#     display(df1)
    
    # Drop NaN values
    df1 = df1.dropna()
    print(df1)

    print(df1.index)
    print(df1.get_value(datetime(2009, 01, 02),'Adj Close'))
#     df = pd.DataFrame(np.random.randn(5,3),columns=list('ABC'))

#     print(df)
#     print(df.iat[0,0])
#     print(df.at[0,'A'])

if __name__ == "__main__": # if run standalone
    test_run()


                  Open        High         Low       Close      Volume  \
2009-01-02  308.600520  321.820563  305.500529  321.320557   7248000.0   
2009-01-05  321.000544  331.240583  315.000528  328.050564   9814500.0   
2009-01-06  332.980573  340.800591  326.390554  334.060566  12898500.0   
2009-01-07  328.320555  330.910553  318.750561  322.010543   9022600.0   
2009-01-08  318.280544  325.190545  317.340539  325.190545   7228300.0   
2009-01-09  327.500567  327.500567  313.400526  315.070553   8713400.0   
2009-01-12  316.310537  318.950558  310.230541  312.690537   6633300.0   
2009-01-13  311.770535  320.600551  310.390532  314.320528   8898100.0   
2009-01-14  310.000525  313.800550  297.750506  300.970513  10976700.0   
2009-01-15  297.570512  303.580514  286.790492  298.990520  11913400.0   
2009-01-16  305.020525  308.250518  295.700519  299.670521  10487900.0   
2009-01-20  299.140525  299.500513  282.750497  282.750497  10134100.0   
2009-01-21  288.350489  303.500534  28

In [60]:
from datetime import timedelta

dfSPY = pd.read_csv('stock_data/SPY.csv', index_col=0)
df1 = pd.DataFrame(index=dfSPY.index)
df2 = pd.read_csv('stock_data/' + 'GOOGL' + '.csv', index_col="Date", parse_dates=True, na_values=['nan'])
df1 = df1.join(df2)
df1 = df1.dropna()
GOOGL = df1
# display(GOOGL)

index = 1
counter = 0
date_list = []

while counter < 10:
    try:
        GOOGL.loc[datetime(2009, 1, 2) + timedelta(days=index)]
    except KeyError:
        print("KeyError occuring...")
        index += 1
    else:
        print("All looking good!")
        date_list.append(datetime(2009, 1, 2) + timedelta(days=index))
        index += 1
        counter += 1

print(date_list)
print(len(date_list))

gMEAN = GOOGL.loc[date_list].mean()
gMEAN.index = ['open_10d_mean', 'high_10d_mean', 'low_10d_mean', 'close_10d_mean', 'volume_10d_mean', 'adjclose_10d_mean']
gMAX = GOOGL.loc[date_list].max()
gMAX.index = ['open_10d_max', 'high_10d_max', 'low_10d_max', 'close_10d_max', 'volume_10d_max', 'adjclose_10d_max']

print(gMEAN)

result = pd.concat([gMEAN, gMAX], axis=1).T

# dfTemp = gMEAN.append(gMAX)
print(result)
# dates = pd.date_range(end='2/08/2014', periods=8)[::-1]
# display(dates)

KeyError occuring...
KeyError occuring...
All looking good!
All looking good!
All looking good!
All looking good!
All looking good!
KeyError occuring...
KeyError occuring...
All looking good!
All looking good!
All looking good!
All looking good!
All looking good!
[datetime.datetime(2009, 1, 5, 0, 0), datetime.datetime(2009, 1, 6, 0, 0), datetime.datetime(2009, 1, 7, 0, 0), datetime.datetime(2009, 1, 8, 0, 0), datetime.datetime(2009, 1, 9, 0, 0), datetime.datetime(2009, 1, 12, 0, 0), datetime.datetime(2009, 1, 13, 0, 0), datetime.datetime(2009, 1, 14, 0, 0), datetime.datetime(2009, 1, 15, 0, 0), datetime.datetime(2009, 1, 16, 0, 0)]
10
open_10d_mean        3.168750e+02
high_10d_mean        3.220820e+02
low_10d_mean         3.091740e+02
close_10d_mean       3.151020e+02
volume_10d_mean      9.613030e+06
adjclose_10d_mean    1.577087e+02
dtype: float64
   adjclose_10d_max  adjclose_10d_mean  close_10d_max  close_10d_mean  \
0               NaN          157.70871            NaN      315.10